In [1]:
class Pulse():
    def __init__(self, source, destination, is_high) -> None:
        self.source = source
        self.destination = destination
        self.is_high = is_high

    def __repr__(self) -> str:
        return f"{self.source.name if self.source else 'Button'} -{'high' if self.is_high else 'low'}-> {self.destination.name}"

In [2]:
class FlipFlop():
    def __init__(self, name, destinations) -> None:
        self.is_on = False
        self.name = name
        self.destinations = destinations
        self.type ="flipflop"

    def handle_pulse(self, pulse: Pulse):
        new_pulses = []
        if not pulse.is_high:
            self.is_on = not self.is_on
            for destination in self.destinations:
                new_pulses.append(Pulse(self, destination, self.is_on))
        return new_pulses

In [3]:
class Broadcaster():
    def __init__(self, name, destinations) -> None:
        self.destinations = destinations
        self.name = name
        self.type ="broadcaster"

    def handle_pulse(self, pulse: Pulse):
        new_pulses = []
        for destination in self.destinations:
            new_pulses.append(Pulse(self, destination, pulse.is_high))
        return new_pulses

In [4]:
class Conjunction():
    def __init__(self, name, destinations) -> None:
        self.destinations = destinations
        self.name = name
        self.inputs = {}
        self.type ="conjunction"

    def register_input(self, source):
        self.inputs[source.name] = False


    def handle_pulse(self, pulse: Pulse):
        new_pulses = []
        self.inputs[pulse.source.name] = pulse.is_high
        all_true = sum((self.inputs.values())) == len(self.inputs)
        for destination in self.destinations:
            new_pulses.append(Pulse(self, destination, not all_true))
        return new_pulses

In [5]:
parsing_items = False
with open("day20.txt") as f:
    lines = f.readlines()
nodes = {}
for i, line in enumerate(lines):
    if len(line) > 3:
        if i != len(lines) -1:
            line = line[:-1]
        #print(line)
        destinations = line.split(">")[1].split(",")
        destinations = [dest.strip() for dest in destinations]
        name = line.split()[0][1:]
        if line.startswith("broadcaster"):
            nodes["broadcaster"] = Broadcaster("broadcaster", destinations)
        elif line.startswith("%"):
            nodes[name] = FlipFlop(name, destinations)
        elif line.startswith("&"):
            nodes[name] = Conjunction(name, destinations)

for node in nodes.values():
    node.destinations = [nodes[dest] if dest in nodes else Broadcaster(dest, []) for dest in node.destinations]
    for destination in node.destinations:
        if destination.type == "conjunction":
            destination.register_input(node)

In [6]:
# there is one conj node before the rx node - with four input. Lets check how often each input cycles on True

In [7]:
from collections import deque
queue = deque()
button_presses = 0
for i in range(1000000):
    queue.append(Pulse(None, nodes["broadcaster"], False))
    button_presses += 1
    conj_node = nodes["ns"].inputs
    while (len(queue)):
        signal = queue.popleft()
        if signal.destination.name == 'rx' and not signal.is_high:
            break
        if signal.destination.name == "ns" and (conj_node["dc"] or conj_node["rv"] or conj_node["vp"] or conj_node["cq"]):         
            print(f'{button_presses=} {nodes["ns"].inputs}')
        new_signals = signal.destination.handle_pulse(signal)
        for new_signal in new_signals:
            queue.append(new_signal)
    if conj_node["dc"] or conj_node["rv"] or conj_node["vp"] or conj_node["cq"]:         
        print(nodes["ns"].inputs)
    if signal.destination.name == 'rx' and not signal.is_high:
            break
button_presses


button_presses=3797 {'dc': True, 'rv': False, 'vp': False, 'cq': False}
button_presses=3847 {'dc': False, 'rv': False, 'vp': True, 'cq': False}
button_presses=3847 {'dc': False, 'rv': False, 'vp': True, 'cq': False}
button_presses=3847 {'dc': False, 'rv': False, 'vp': True, 'cq': False}
button_presses=3877 {'dc': False, 'rv': False, 'vp': False, 'cq': True}
button_presses=3877 {'dc': False, 'rv': False, 'vp': False, 'cq': True}
button_presses=3877 {'dc': False, 'rv': False, 'vp': False, 'cq': True}
button_presses=3877 {'dc': False, 'rv': False, 'vp': False, 'cq': True}
button_presses=3877 {'dc': False, 'rv': False, 'vp': False, 'cq': True}
button_presses=4051 {'dc': False, 'rv': True, 'vp': False, 'cq': False}
button_presses=4051 {'dc': False, 'rv': True, 'vp': False, 'cq': False}
button_presses=4051 {'dc': False, 'rv': True, 'vp': False, 'cq': False}
button_presses=4051 {'dc': False, 'rv': True, 'vp': False, 'cq': False}
button_presses=7594 {'dc': True, 'rv': False, 'vp': False, 'cq':

1000000

In [8]:
cycle_dc = 3797
cycle_rv = 4051
cycle_vp = 3847
cycle_cq = 3877

In [9]:
from math import lcm
lcm(*[cycle_dc, cycle_rv, cycle_vp, cycle_cq])

229414480926893